In [ ]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
tables = soup.find_all('table',class_='wikitable sortable')

In [ ]:
len(tables)

In [ ]:
type(tables[0])

In [ ]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [ ]:
df

In [ ]:
df_2020 = df[['Title','Cast and crew']]

In [ ]:
df_2020

In [ ]:
!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '8c07c60b441f06b53537062ed576687b'

In [ ]:
from tmdbv3api import Movie
import time
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      time.sleep(3)
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
    
      else:
          return np.NaN

In [ ]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2020

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [ ]:
new_df20

In [ ]:
new_df20.to_csv('F:\\data_2020.csv',index=False)

In [ ]:
df_2016_17 = pd.read_csv('data_2016-17.csv')
df_2016_17

In [ ]:
df_2018_19 = pd.read_csv('data_2018-19.csv')
df_2018_19

In [ ]:
df_2020 = pd.read_csv('data_2020.csv')
df_2020

In [ ]:
upto_2019 = df_2016_17.append(df_2018_19,ignore_index=True)
upto_2019

In [ ]:
final_data=upto_2019.append(df_2020,ignore_index=True)
final_data

In [ ]:
final_data.to_csv('main_data.csv',index=False)